In [ ]:
import pandas as pd

file_path = 'refined_comprehensive_dataset.xlsx'
excel_data = pd.ExcelFile(file_path)

first_sheet = excel_data.parse(sheet_name=0)
print(first_sheet.columns)


Index(['Event Name', 'Gender', 'Time', 'OutfitType', 'Dress Type',
       'Dress Color', 'Dress Fabric/Texture', 'Shoes Type', 'Shoes Color',
       'Upper layer', 'Upper layer color', 'Weather Range'],
      dtype='object')


In [ ]:
import pandas as pd


file_path = '/content/dataset.xlsx'
data = pd.ExcelFile(file_path)
first_sheet = data.parse(data.sheet_names[0])

def enhanced_rule_based_filtering(weather_range, gender, event_name, outfit_type, time):
    """
    Filters dress recommendations based on weather, gender, event name, outfit type, and time.

    Parameters:
        weather_range (str): Weather condition, e.g., "0-10".
        gender (str): Gender, e.g., "Male", "Female".
        event_name (str): Event name, e.g., "Aqiqa".
        outfit_type (str): Outfit type, e.g., "Formal", "Casual".
        time (str): Time of the day, e.g., "Day", "Night".

    Returns:
        DataFrame: Filtered recommendations.
    """
    # Filter dataset based on criteria
    filtered_data = first_sheet[
        (first_sheet['Weather Range'].str.contains(weather_range, na=False)) &
        (first_sheet['Gender'].str.contains(gender, na=False)) &
        (first_sheet['Event Name'].str.contains(event_name, na=False)) &
        (first_sheet['OutfitType'].str.contains(outfit_type, na=False)) &
        (first_sheet['Time'].str.contains(time, na=False))
    ].copy()

    # Relevant columns for suggestions
    suggestion_columns = [
        'OutfitType', 'Event Name', 'Dress Type', 'Dress Color',
        'Shoes Type', 'Shoes Color',
        'Upper layer', 'Upper layer color', 'Dress Fabric/Texture'
    ]

    return filtered_data[suggestion_columns]

# Example Input
weather = "0-10"
gender = "Male"
event = "Aqiqa"
outfit = "Formal"
time_of_day = "Night"

rule_based_suggestions = enhanced_rule_based_filtering(weather, gender, event, outfit, time_of_day)

print(rule_based_suggestions.head())

rule_based_suggestions.to_excel('/content/rule_based_recommendations.xlsx', index=False)
print("Recommendations saved to 'rule_based_recommendations.xlsx'")


  OutfitType Event Name                       Dress Type  \
9     Formal      Aqiqa  Shalwar-Kameez,  Kurta-Pajama,    

                        Dress Color                           Shoes Type  \
9  Black, Burgundy, Navy Blue, Skin  Chukka Boots, Leather sheo, Oxford    

         Shoes Color               Upper layer        Upper layer color  \
9  Black, Dark Brown  Blazer, long-coat, Shawl  Black,brown,Grey,Golden   

                  Dress Fabric/Texture  
9  Boski, Wool-Blend, Khaddar, Karandi  
Recommendations saved to 'rule_based_recommendations.xlsx'


In [ ]:
import pandas as pd

file_path = '/content/rule_based_recommendations.xlsx'
rule_based_df = pd.read_excel(file_path)

for _, row in rule_based_df.iterrows():
    print("Row data:", row.to_dict())
    outfit_type = row.get('OutfitType', '')
    print("Matched Outfit Type:", outfit_type)


Row data: {'OutfitType': 'Formal', 'Event Name': 'Aqiqa', 'Dress Type': 'Shalwar-Kameez,  Kurta-Pajama, ', 'Dress Color': 'Black, Burgundy, Navy Blue, Skin', 'Shoes Type': 'Chukka Boots, Leather sheo, Oxford ', 'Shoes Color': 'Black, Dark Brown', 'Upper layer': 'Blazer, long-coat, Shawl', 'Upper layer color': 'Black,brown,Grey,Golden', 'Dress Fabric/Texture': 'Boski, Wool-Blend, Khaddar, Karandi'}
Matched Outfit Type: Formal


In [ ]:
import pandas as pd
from itertools import product, combinations
import random

def split_and_strip(s):
    if not isinstance(s, str) or not s.strip():
        return []
    return [x.strip() for x in s.split(',') if x.strip()]

def intersect_lists(rule_list, user_list):
    rule_set = set(x.lower() for x in rule_list)
    user_set = set(x.lower() for x in user_list)
    intersection = rule_set.intersection(user_set)
    # Keep original case from rule_list for consistent output
    return [x for x in rule_list if x.lower() in intersection]

def outfits_all_attributes_different(o1, o2):
    # Check if all attributes differ between two outfits
    for a1, a2 in zip(o1, o2):
        if a1.lower() == a2.lower():
            return False
    return True

def max_distinct_outfits(all_outfits, max_outfits=3):
    # Try to find max_outfits outfits all mutually different in all attributes
    for combo in combinations(all_outfits, max_outfits):
        all_diff = True
        for i in range(len(combo)):
            for j in range(i+1, len(combo)):
                if not outfits_all_attributes_different(combo[i], combo[j]):
                    all_diff = False
                    break
            if not all_diff:
                break
        if all_diff:
            return list(combo)
    # If no fully different combos found, return as many distinct as possible
    distinct_list = []
    for outfit in all_outfits:
        if all(outfits_all_attributes_different(outfit, o) for o in distinct_list):
            distinct_list.append(outfit)
        if len(distinct_list) == max_outfits:
            break
    if distinct_list:
        return distinct_list
    return all_outfits[:max_outfits]

def regenerate_all_attributes(dress_types, dress_colors, shoe_types, shoe_colors, upper_layers, upper_colors, fabrics):
    # Randomly shuffle and select types/colors for each category if multiple available
    new_dress_types = random.sample(dress_types, len(dress_types)) if len(dress_types) > 1 else dress_types
    new_shoe_types = random.sample(shoe_types, len(shoe_types)) if len(shoe_types) > 1 else shoe_types
    new_upper_layers = random.sample(upper_layers, len(upper_layers)) if len(upper_layers) > 1 else upper_layers

    new_dress_colors = random.sample(dress_colors, len(dress_colors)) if len(dress_colors) > 1 else dress_colors
    new_shoe_colors = random.sample(shoe_colors, len(shoe_colors)) if len(shoe_colors) > 1 else shoe_colors
    new_upper_colors = random.sample(upper_colors, len(upper_colors)) if len(upper_colors) > 1 else upper_colors

    # Handling the fabric/texture attribute
    new_fabrics = random.sample(fabrics, len(fabrics)) if len(fabrics) > 1 else fabrics

    return new_dress_types, new_dress_colors, new_shoe_types, new_shoe_colors, new_upper_layers, new_upper_colors, new_fabrics

def enhanced_rule_based_filtering(rule_data, weather_range, gender, event_name, outfit_type, time_of_day):
    filtered = rule_data[
        (rule_data['Weather Range'].str.contains(weather_range, na=False)) &  # Ensure weather range filtering is correct
        (rule_data['Gender'].str.contains(gender, na=False)) &  # Ensure gender filtering is correct
        (rule_data['Event Name'].str.contains(event_name, na=False)) &  # Event name filter
        (rule_data['OutfitType'].str.contains(outfit_type, na=False)) &  # Outfit type filter
        (rule_data['Time'].str.contains(time_of_day, na=False))  # Time filter
    ].copy()
    return filtered

def recommend_outfits_with_feedback(rule_data, weather, gender, event, outfit, time_of_day, user_wardrobe, max_retries=5):
    filtered_rules = enhanced_rule_based_filtering(rule_data, weather, gender, event, outfit, time_of_day)
    if filtered_rules.empty:
        print("No rule-based recommendations found.")
        return

    for idx, rule_row in filtered_rules.iterrows():
        retries = 0

        # Intersect types and colors with user wardrobe and rule
        dress_types = intersect_lists(split_and_strip(rule_row['Dress Type']), user_wardrobe['Dress Type'])
        dress_colors = intersect_lists(split_and_strip(rule_row['Dress Color']), user_wardrobe['Dress Color'])
        shoe_types = intersect_lists(split_and_strip(rule_row['Shoes Type']), user_wardrobe['Shoes Type'])
        shoe_colors = intersect_lists(split_and_strip(rule_row['Shoes Color']), user_wardrobe['Shoes Color'])
        upper_layers = intersect_lists(split_and_strip(rule_row['Upper layer']), user_wardrobe['Upper layer'])
        upper_colors = intersect_lists(split_and_strip(rule_row['Upper layer color']), user_wardrobe['Upper layer color'])

        # Intersect the fabric/texture with user wardrobe
        fabrics = intersect_lists(split_and_strip(rule_row['Dress Fabric/Texture']), user_wardrobe.get('Dress Fabric/Texture', []))

        if not (dress_types and dress_colors and shoe_types and shoe_colors and upper_layers and upper_colors):
            continue

        while retries < max_retries:
            # Regenerate all types and colors (shuffle) if multiple options available
            new_dress_types, new_dress_colors, new_shoe_types, new_shoe_colors, new_upper_layers, new_upper_colors, new_fabrics = regenerate_all_attributes(
                dress_types, dress_colors, shoe_types, shoe_colors, upper_layers, upper_colors, fabrics
            )

            # Generate all combinations including fabrics
            all_outfits = list(product(
                new_dress_types,
                new_dress_colors,
                new_shoe_types,
                new_shoe_colors,
                new_upper_layers,
                new_upper_colors,
                new_fabrics
            ))

            # Pick up to 3 outfits with max difference
            selected = max_distinct_outfits(all_outfits, max_outfits=3)

            if not selected:
                print("Could not generate any valid outfit.")
                return

            df = pd.DataFrame(selected, columns=['Dress Type', 'Dress Color', 'Shoes Type', 'Shoes Color', 'Upper layer', 'Upper layer color', 'Dress Fabric/Texture'])
            print(f"\nAttempt {retries + 1}: Here are 3 outfit recommendations:")
            print(df)

            df.to_excel("recommended_outfits.xlsx", index=False)
            print("Recommendations saved to 'recommended_outfits.xlsx'")

            feedback = input("Do you accept these recommendations? (yes/no): ").strip().lower()
            if feedback == 'yes':
                print("Great! Recommendations accepted.")
                return df
            else:
                print("Regenerating all outfit attributes (types & colors)...\n")
                retries += 1

        print(f"Max retries ({max_retries}) reached. Returning last recommendations.")
        return df

    print("No suitable outfits matched user wardrobe and rules.")


# --- Example usage ---
file_path = '/content/dataset.xlsx'  # Replace with your actual file path
xls = pd.ExcelFile(file_path)
rule_data = xls.parse(xls.sheet_names[0])

user_wardrobe = {
    'Dress Type': ['Shalwar-Kameez', 'Kurta-Pajama', 'Jeans', 'Trousers'],
    'Dress Color': ['Black', 'Navy Blue', 'White', 'Beige', 'Skin'],
    'Shoes Type': ['Oxford', 'Leather Shoe', 'Chukka Boots'],
    'Shoes Color': ['Black', 'Dark Brown', 'Beige'],
    'Upper layer': ['Blazer', 'Long-coat', 'Shawl'],
    'Upper layer color': ['Black', 'Brown', 'Grey'],
    'Dress Fabric/Texture': ['Boski', 'Wool-Blend', 'Khaddar', 'Karandi']  # Added fabrics
}

weather = "0-10"
gender = "Male"
event = "Aqiqa"
outfit = "Formal"
time_of_day = "Night"

recommend_outfits_with_feedback(rule_data, weather, gender, event, outfit, time_of_day, user_wardrobe)



Attempt 1: Here are 3 outfit recommendations:
       Dress Type Dress Color    Shoes Type Shoes Color Upper layer  \
0  Shalwar-Kameez   Navy Blue  Chukka Boots  Dark Brown   long-coat   
1    Kurta-Pajama        Skin        Oxford       Black      Blazer   

  Upper layer color Dress Fabric/Texture  
0             brown           Wool-Blend  
1              Grey                Boski  
Recommendations saved to 'recommended_outfits.xlsx'
Do you accept these recommendations? (yes/no): no
Regenerating all outfit attributes (types & colors)...


Attempt 2: Here are 3 outfit recommendations:
       Dress Type Dress Color    Shoes Type Shoes Color Upper layer  \
0  Shalwar-Kameez   Navy Blue  Chukka Boots  Dark Brown   long-coat   
1    Kurta-Pajama        Skin        Oxford       Black       Shawl   

  Upper layer color Dress Fabric/Texture  
0              Grey              Khaddar  
1             brown           Wool-Blend  
Recommendations saved to 'recommended_outfits.xlsx'
Do you acc

,Dress Type,Dress Color,Shoes Type,Shoes Color,Upper layer,Upper layer color,Dress Fabric/Texture
0,Shalwar-Kameez,Navy Blue,Chukka Boots,Dark Brown,long-coat,Grey,Khaddar
1,Kurta-Pajama,Skin,Oxford,Black,Shawl,brown,Wool-Blend
